In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# https://pythonhosted.org/pywebhdfs/

In [2]:
import pyspark
from pyspark.ml.feature import Word2Vec
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.ml import Pipeline, PipelineModel
import os
 


In [3]:
os.environ['PYSPARK_PYTHON'] = '/home/hduser/anaconda3/bin/python'
# The value is the python command of the version required to start the master and worker in the Linux system
os.environ['PYSPARK_DRIVER_PYTHON'] = r"C:\Users\billy\anaconda3\python.exe"
# The value is the spark directory in the local windows system
os.environ['SPARK_HOME'] = 'C:/spark'
# The value is the local IP, and the IP required to establish a connection, to prevent connection failure when multiple network cards
#os.environ['SPARK_LOCAL_IP'] = '192.168.56.1'
os.environ['HADOOP_HOME'] = "D:/hadoop-3.3.0"
os.environ['HADOOP_CONF_DIR'] = "/usr/local/hadoop/etc/hadoop"

In [4]:
conf  = pyspark.SparkConf().setAppName('forest').setMaster('spark://192.168.133.4:7077').set(
    "spark.submit.deployMode","client").set('spark.driver.memory','2g').set(
        'spark.executor.memory', '2g').set('spark.executor.cores', 1).set(
        'spark.network.timeout', 600).set('spark.executor.heartbeatInterval', 120).set(
    'spark.cores.max', 4).set("spark.driver.host","192.168.133.1").set("spark.driver.port","9999").set("spark.sql.execution.arrow.pyspark.enabled", "true")#.set('spark.python.profile','true')
sc = pyspark.SparkContext(conf=conf)

In [5]:
path = "hdfs://192.168.133.4:9000/user/hduser/"

In [6]:
RawUserRDD = sc.textFile(path+"data/covtype.data")
print(RawUserRDD.count())

581012


In [7]:
RawData = RawUserRDD.map(lambda x:x.split(","))
print(RawData.take(1),sep='')

[['2596', '51', '3', '258', '0', '510', '221', '232', '148', '6279', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '5']]


In [8]:
from pyspark.mllib.regression import LabeledPoint

In [14]:
(trainData, validationData, testData) = PrepareData(sc)
trainData.cache()
validationData.cache()
testData.cache()

total_row:581012
trainData:464590, validationData:58351, testData:58071


PythonRDD[10] at RDD at PythonRDD.scala:53

PythonRDD[11] at RDD at PythonRDD.scala:53

PythonRDD[12] at RDD at PythonRDD.scala:53

In [15]:
from pyspark.mllib.tree import DecisionTree
model = DecisionTree.trainClassifier(trainData,
                    numClasses=7, categoricalFeaturesInfo={},
                    impurity="entropy",
                    maxDepth=8,
                    maxBins=5)


In [16]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [17]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [18]:
accuracy = evaluateModel(model, validationData)

In [19]:
def PrepareData(sc):
    rawData = sc.textFile(path+"data/covtype.data")
    print('total_row:{}'.format(rawData.count() ))
    lines = rawData.map(lambda x:x.split(","))
    labelpointRDD = lines.map(lambda r:LabeledPoint(
                            extract_label(r),
                            extract_features(r,len(r)-1)))
    (trainData, validationData, testData) = labelpointRDD.randomSplit([8, 1, 1])
    print('trainData:{}, validationData:{}, testData:{}'.format(
                trainData.count(),validationData.count(),testData.count()))
    return (trainData, validationData, testData)
    
def extract_label(record):
    label = (record[-1])
    return float(label)-1

def extract_features(record,featureEnd):
    numericalFeatures = [convert_float(field) for field in record[0: featureEnd]]
    return numericalFeatures

def convert_float(x):
    return (0 if x=="?" else float(x))

In [20]:
def trainEvaluateModel(trainData, validationData, impurityParm, maxDepthParm, maxBinsParm):
    model = DecisionTree.trainClassifier(trainData,
                        numClasses=7, categoricalFeaturesInfo={},
                        impurity=impurityParm,
                        maxDepth=maxDepthParm,
                        maxBins=maxBinsParm)
    accuracy = evaluateModel(model, validationData)

    return (accuracy,impurityParm,maxDepthParm,maxBinsParm,model)

In [21]:
def evaluateModel(model, validationData):
    score = model.predict(validationData.map(lambda p:p.features))
    scoreAndLabels = score.zip(validationData.map(lambda p:p.label))
    metrics = MulticlassMetrics(scoreAndLabels)
    accuracy = metrics.accuracy
    
    return accuracy

In [22]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [23]:
model = DecisionTreeClassifier(trainData, categoricalFeaturesInfo={}, impurity='variance', numClasses=2)

paramGrid = ParamGridBuilder() \
    .addGrid(model.maxDepth, [4, 5, 6, 7]) \
    .addGrid(model.maxBins, [24, 28, 32, 36]) \
    .build()

# crossval = CrossValidator(estimator=model,
#                           estimatorParamMaps=paramGrid,
#                           evaluator=BinaryClassificationEvaluator(),
#                           numFolds=3)  

# # Run cross-validation, and choose the best set of parameters.
# cvModel = crossval.fit(training)

TypeError: Method __init__ forces keyword arguments.

In [24]:
model  = DecisionTree.trainClassifier(trainData, categoricalFeaturesInfo={},  numClasses=7)


In [25]:
model  = DecisionTree.trainClassifier(trainData, categoricalFeaturesInfo={},  numClasses=7)

# We use a ParamGridBuilder to construct a grid of parameters to search over.
# TrainValidationSplit will try all combinations of values and determine best model using
# the evaluator.

paramGrid = ParamGridBuilder() \
    .addGrid(model.depth, [4, 5, 6, 7]) \
    .build()
# In this case the estimator is simply the linear regression.
# A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
tvs = CrossValidator(estimator=DTree,
                           estimatorParamMaps=paramGrid,
                           evaluator=MulticlassClassificationEvaluator(),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8,
                    numFolds = 5)

# Run TrainValidationSplit, and choose the best set of parameters.
model = tvs.fit(trainData)

# Make predictions on test data. model is the model with combination of parameters
# that performed best.
model.transform(testData)\
    .select("features", "label", "prediction")\
    .show()

TypeError: param must be an instance of Param

In [26]:
for lp in labelpointRDD.take(100):
    predict = model.predict(lp.features)
    label = lp.label
    features = lp.features
    result = ("correct" if (label ==predict) else "error")
    print("土地條件:海拔:" + str(features[0])+
         " 方位: " + str(features[1]) +
         " 斜率: " + str(features[2]) +
         " 水源垂直距離: " + str(features[3]) +
         " 水源水平距離: " + str(features[4]) +
         " 9點時陰影: " + str(features[5]) +
         " ...==>預測: " + str(predict) +
         " 實際: " + str(label)+"結果:"+result)

NameError: name 'labelpointRDD' is not defined

# Pipeline

In [27]:
from pyspark.sql.types import StringType,StructField,StructType

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [29]:
rawData = sc.textFile(path+'data/covtype.data')
lines = rawData.map(lambda x:x.split(","))
lines.count()

581012

In [30]:
field_num = len(lines.first())

In [31]:
fields = [StructField("field"+str(i),StringType(),True)
          for i in range(field_num)]

In [32]:
schema = StructType(fields)

In [33]:
schema

StructType(List(StructField(field0,StringType,true),StructField(field1,StringType,true),StructField(field2,StringType,true),StructField(field3,StringType,true),StructField(field4,StringType,true),StructField(field5,StringType,true),StructField(field6,StringType,true),StructField(field7,StringType,true),StructField(field8,StringType,true),StructField(field9,StringType,true),StructField(field10,StringType,true),StructField(field11,StringType,true),StructField(field12,StringType,true),StructField(field13,StringType,true),StructField(field14,StringType,true),StructField(field15,StringType,true),StructField(field16,StringType,true),StructField(field17,StringType,true),StructField(field18,StringType,true),StructField(field19,StringType,true),StructField(field20,StringType,true),StructField(field21,StringType,true),StructField(field22,StringType,true),StructField(field23,StringType,true),StructField(field24,StringType,true),StructField(field25,StringType,true),StructField(field26,StringType,t

In [34]:
covtype_df = spark.createDataFrame(lines,schema)

Exception ignored in: <function JavaWrapper.__del__ at 0x000001B3242DE4C0>
Traceback (most recent call last):
  File "C:\Users\billy\anaconda3\envs\tensorflow-gpu\lib\site-packages\pyspark\ml\wrapper.py", line 42, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'DecisionTreeClassifier' object has no attribute '_java_obj'


In [35]:
from pyspark.sql.functions import col
covtype_df = covtype_df.select([ col(column).cast("double").alias(column)
                                for column in covtype_df.columns])

In [36]:
covtype_df.printSchema()

root
 |-- field0: double (nullable = true)
 |-- field1: double (nullable = true)
 |-- field2: double (nullable = true)
 |-- field3: double (nullable = true)
 |-- field4: double (nullable = true)
 |-- field5: double (nullable = true)
 |-- field6: double (nullable = true)
 |-- field7: double (nullable = true)
 |-- field8: double (nullable = true)
 |-- field9: double (nullable = true)
 |-- field10: double (nullable = true)
 |-- field11: double (nullable = true)
 |-- field12: double (nullable = true)
 |-- field13: double (nullable = true)
 |-- field14: double (nullable = true)
 |-- field15: double (nullable = true)
 |-- field16: double (nullable = true)
 |-- field17: double (nullable = true)
 |-- field18: double (nullable = true)
 |-- field19: double (nullable = true)
 |-- field20: double (nullable = true)
 |-- field21: double (nullable = true)
 |-- field22: double (nullable = true)
 |-- field23: double (nullable = true)
 |-- field24: double (nullable = true)
 |-- field25: double (nullable

In [37]:
featuresCols=covtype_df.columns[:54]
print(featuresCols)

['field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7', 'field8', 'field9', 'field10', 'field11', 'field12', 'field13', 'field14', 'field15', 'field16', 'field17', 'field18', 'field19', 'field20', 'field21', 'field22', 'field23', 'field24', 'field25', 'field26', 'field27', 'field28', 'field29', 'field30', 'field31', 'field32', 'field33', 'field34', 'field35', 'field36', 'field37', 'field38', 'field39', 'field40', 'field41', 'field42', 'field43', 'field44', 'field45', 'field46', 'field47', 'field48', 'field49', 'field50', 'field51', 'field52', 'field53']


In [38]:
covtype_df = covtype_df.withColumn("label",covtype_df["field54"]-1).drop("field54")

In [39]:
covtype_df.show(1)

+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+
|field0|field1|field2|field3|field4|field5|field6|field7|field8|field9|field10|field11|field12|field13|field14|field15|field16|field17|field18|field19|field20|field21|field22|field23|field24|field25|field26|field27|field28|field29|field30|field31|field32|field33|field34|field35|field36|field37|field38|field39|field40|field41|field42|field43|field44|field45|field46|field47|field48|field49|field50|field51|field52|field53|label|
+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-----

In [40]:
train_df, test_df = covtype_df.randomSplit([0.7,0.3])
train_df.cache()
test_df.cache()

DataFrame[field0: double, field1: double, field2: double, field3: double, field4: double, field5: double, field6: double, field7: double, field8: double, field9: double, field10: double, field11: double, field12: double, field13: double, field14: double, field15: double, field16: double, field17: double, field18: double, field19: double, field20: double, field21: double, field22: double, field23: double, field24: double, field25: double, field26: double, field27: double, field28: double, field29: double, field30: double, field31: double, field32: double, field33: double, field34: double, field35: double, field36: double, field37: double, field38: double, field39: double, field40: double, field41: double, field42: double, field43: double, field44: double, field45: double, field46: double, field47: double, field48: double, field49: double, field50: double, field51: double, field52: double, field53: double, label: double]

DataFrame[field0: double, field1: double, field2: double, field3: double, field4: double, field5: double, field6: double, field7: double, field8: double, field9: double, field10: double, field11: double, field12: double, field13: double, field14: double, field15: double, field16: double, field17: double, field18: double, field19: double, field20: double, field21: double, field22: double, field23: double, field24: double, field25: double, field26: double, field27: double, field28: double, field29: double, field30: double, field31: double, field32: double, field33: double, field34: double, field35: double, field36: double, field37: double, field38: double, field39: double, field40: double, field41: double, field42: double, field43: double, field44: double, field45: double, field46: double, field47: double, field48: double, field49: double, field50: double, field51: double, field52: double, field53: double, label: double]

In [41]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier


In [42]:
vectorAssembler = VectorAssembler(inputCols=featuresCols,
                                 outputCol = "features")
dt = DecisionTreeClassifier(labelCol="label",featuresCol="features",
                           maxDepth=5,maxBins=20)

In [43]:
dt_pipeline = Pipeline(stages=[vectorAssembler,dt])

In [44]:
dt_pipeline.getStages()

[VectorAssembler_c081158e6aec, DecisionTreeClassifier_bf7b19909585]

In [45]:
pipelineModel = dt_pipeline.fit(train_df)

In [46]:
predicted = pipelineModel.transform(test_df)

In [47]:
print(predicted.columns,sep='')

['field0', 'field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7', 'field8', 'field9', 'field10', 'field11', 'field12', 'field13', 'field14', 'field15', 'field16', 'field17', 'field18', 'field19', 'field20', 'field21', 'field22', 'field23', 'field24', 'field25', 'field26', 'field27', 'field28', 'field29', 'field30', 'field31', 'field32', 'field33', 'field34', 'field35', 'field36', 'field37', 'field38', 'field39', 'field40', 'field41', 'field42', 'field43', 'field44', 'field45', 'field46', 'field47', 'field48', 'field49', 'field50', 'field51', 'field52', 'field53', 'label', 'features', 'rawPrediction', 'probability', 'prediction']


In [48]:
predicted.select('features', 'rawPrediction', 'probability', 'prediction').show(10)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,324.0,10268....|[0.0,0.0206527281...|       2.0|
|(54,[0,1,2,5,6,7,...|[0.0,31.0,395.0,7...|[0.0,0.0234848484...|       3.0|
+-----------

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
evaluator = MulticlassClassificationEvaluator(
                labelCol="label",predictionCol="prediction",
                metricName="accuracy"
                )

In [51]:
predictions = pipelineModel.transform(test_df)
accuracy = evaluator.evaluate(predictions)
accuracy

0.7019220862477533

In [52]:
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit

In [53]:
paramGrid = ParamGridBuilder()\
        .addGrid(dt.impurity,['gini','entropy'])\
        .addGrid(dt.maxDepth, [10,15,25])\
        .addGrid(dt.maxBins,[30,40,50])\
        .build()

In [54]:
tvs = TrainValidationSplit(estimator = dt, evaluator= evaluator,
                          estimatorParamMaps=paramGrid,trainRatio=0.8)

In [55]:
tvs_pipeline = Pipeline(stages = [vectorAssembler,tvs])

In [56]:
tvs_pipelineModel = tvs_pipeline.fit(train_df)

In [57]:
bestModel = tvs_pipelineModel.stages[1].bestModel
print(bestModel.toDebugString[:500])

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_bf7b19909585, depth=25, numNodes=44125, numClasses=7, numFeatures=54
  If (feature 0 <= 2693.5)
   If (feature 0 <= 2512.5)
    If (feature 0 <= 2413.5)
     If (feature 23 <= 0.5)
      If (feature 3 <= 15.0)
       If (feature 13 <= 0.5)
        If (feature 9 <= 574.5)
         If (feature 16 <= 0.5)
          Predict: 5.0
         Else (feature 16 > 0.5)
          If (feature 5 <= 380.5)
           Predict: 5.0
          Else (featur


In [58]:
predictions = tvs_pipelineModel.transform(test_df)
result = predictions.withColumnRenamed("field0","海拔")\
                    .withColumnRenamed("field1","方位")\
                    .withColumnRenamed("field2","斜率")\
                    .withColumnRenamed("field3","垂直距離")\
                    .withColumnRenamed("field4","水平距離")\
                    .withColumnRenamed("field5","陰影")
result.select("海拔","方位","斜率","垂直距離","水平距離","陰影",'label','prediction').show(10)

+------+-----+----+--------+--------+-----+-----+----------+
|  海拔| 方位|斜率|垂直距離|水平距離| 陰影|label|prediction|
+------+-----+----+--------+--------+-----+-----+----------+
|1861.0| 35.0|14.0|    60.0|    11.0| 85.0|  2.0|       2.0|
|1866.0| 23.0|14.0|    85.0|    16.0|108.0|  2.0|       2.0|
|1872.0| 27.0|21.0|   108.0|    30.0| 67.0|  5.0|       5.0|
|1876.0| 25.0|17.0|   124.0|    26.0|150.0|  2.0|       2.0|
|1876.0| 29.0|19.0|   124.0|    34.0| 90.0|  5.0|       5.0|
|1877.0| 28.0|22.0|   127.0|    35.0| 85.0|  5.0|       5.0|
|1880.0| 35.0|21.0|   150.0|    32.0|150.0|  2.0|       2.0|
|1880.0| 38.0|21.0|   150.0|    38.0|120.0|  5.0|       5.0|
|1882.0| 34.0|21.0|    85.0|    28.0|108.0|  2.0|       2.0|
|1882.0|330.0|18.0|     0.0|     0.0|120.0|  5.0|       5.0|
+------+-----+----+--------+--------+-----+-----+----------+
only showing top 10 rows

